### Traiter au niveau du corpus

In [1]:
import glob
from transformers import pipeline, TokenClassificationPipeline, CamembertTokenizer
from itertools import chain

# === Initialisation du modèle ===
model_name = "Jean-Baptiste/camembert-ner-with-dates"
tokenizer = CamembertTokenizer.from_pretrained(model_name)
nlp: TokenClassificationPipeline = pipeline(
    "ner",
    model=model_name,
    tokenizer=tokenizer,
    aggregation_strategy="simple",
    device=-1
)

# === Fonction de découpage ===
def chunk_text(text, chunk_size=512, overlap=50):
    start = 0
    while start < len(text):
        end = start + chunk_size
        yield text[start:end]
        start += chunk_size - overlap

# === Extraction d'entités sur un grand texte ===
def extraire_entites(texte, nlp):
    chunks = list(chunk_text(texte))
    all_ner_results = list(chain.from_iterable([nlp(chunk) for chunk in chunks]))
    return [
        {"mot": ent["word"], "type": ent["entity_group"]}
        for ent in all_ner_results
    ]

# === Étape 1 : concaténer tout le corpus en une seule chaîne ===
corpus_global = ""

for path_fichier in glob.glob("sample_data/Corpus/*.txt"):
    with open(path_fichier, "r", encoding="utf-8") as f:
        texte = f.read().strip()
        corpus_global += texte + "\n"  # Ajouter un retour à la ligne entre les fichiers

print("Tous les fichiers ont été fusionnés dans un seul texte.")
print(f"Taille totale du corpus (caractères) : {len(corpus_global)}")

# === Étape 2 : extraction d'entités sur l'ensemble du corpus ===
entites_globales = extraire_entites(corpus_global, nlp)
tokens = tokenizer.tokenize(corpus_global)
print(f"Nombre total de tokens (subwords) : {len(tokens)}")
print(f"Nombre total d'entités : {len(entites_globales)}")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/423 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/811k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/210 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/970 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Device set to use cpu


Tous les fichiers ont été fusionnés dans un seul texte.
Taille totale du corpus (caractères) : 1364886
Nombre total de tokens (subwords) : 375162
Nombre total d'entités : 9172


In [2]:
import json
resultat = {"entites": entites_globales}
with open("tous_entites_nommes_Camembert.json", "w", encoding="utf-8") as f:
    json.dump(resultat, f, indent=4, ensure_ascii=False)